<a href="https://www.kaggle.com/code/vincemarcs/mvsa-fusion-models-ml?scriptVersionId=101315992" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install keras-self-attention

  Preparing metadata (setup.py) ... - done
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18912 sha256=79dc71ec196a873a8fc34fe9040fd536fc640bb883542f6fe72caed8c48c0527
  Stored in directory: /root/.cache/pip/wheels/95/b1/a8/5ee00cc137940b2f6fa198212e8f45d813d0e0d9c3a04035a3
Successfully built keras-self-attention


In [2]:
SEED = 61

import os
import re
import gc
import h5py
import torch
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import random as python_random
import tensorflow_addons as tfa
import matplotlib.pyplot as plt

from tqdm import tqdm
from nltk import tokenize
from IPython.display import display_html
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from keras_self_attention import SeqSelfAttention
from transformers import BertTokenizer, BertForMaskedLM, BertModel
from tensorflow.python.keras.layers import Layer, InputSpec, Lambda

from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import RepeatedKFold, KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report

from keras import backend as K
from keras import initializers,regularizers,constraints
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Reshape, Input, Embedding, Flatten, Dense, Dropout, BatchNormalization, Activation #, merge
from keras.layers import TimeDistributed, LSTM, GRU, Bidirectional, Convolution1D, MaxPooling1D, MaxPooling2D, GlobalMaxPooling1D
from keras.layers.core import RepeatVector #, Reshape
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import Sequential, Model, load_model

def reset_seeds():
    np.random.seed(SEED) 
    python_random.seed(SEED)
    tf.random.set_seed(SEED)
    os.environ["PYTHONHASHSEED"] = str(SEED)

# from tensorflow.keras import Model
# from attention import Attention_input1, Attention_input2
# from keras.optimizers import SGD, RMSprop, Adagrad

In [3]:
def read_hdf5(path):
    read_file = h5py.File(path, 'r')
    
    feature_names = list(read_file.keys())
    loaded_data = []
    
    for name in feature_names:
        dataset = read_file[name][:]
        if dataset.dtype == np.dtype('object'):
            dataset = np.array([x.decode('UTF-8') for x in dataset])            
        loaded_data.append((name, dataset))

    return loaded_data

def loadz(path):
    data = np.load(path)['arr_0']
    return data

In [4]:
def load_labels(path):
    data = read_hdf5(path)

    for x in data:
        if x[0] == 'multimodal-labels':
            labels = x[1]
        if x[0] == 'text-labels':
            text_labels = x[1]
        if x[0] == 'image-labels':
            image_labels = x[1]

    return labels, text_labels, image_labels

def merge_mvsa(mvsa_single, mvsa_multiple):
    mvsa = np.concatenate((mvsa_single, mvsa_multiple), axis=0)
    return mvsa

def load_mvsa_feature(feature_name, merge=False):
    folder_path = os.path.join('../input/mvsa-features/', feature_name)
    single_file = 'mvsa-single-{}.npz'.format(feature_name)
    multiple_file = 'mvsa-multiple-{}.npz'.format(feature_name)
    mvsa_single = loadz(os.path.join(folder_path, single_file))
    mvsa_multiple = loadz(os.path.join(folder_path, multiple_file))
    
    if merge == True:
        return merge_mvsa(mvsa_single, mvsa_multiple)
    
    return mvsa_single, mvsa_multiple

In [5]:
def get_features(feature_names):
    mvsa_single_features = []
    mvsa_multiple_features = []

    for name in feature_names:
        
        name_split = name.split('-')
        textual = name_split[1]
        visual = name_split[0]
        
        if textual == 'bert':
            textual = 'bert-base'

        textual_features = load_mvsa_feature(textual)
        visual_features = load_mvsa_feature(visual)

        if 'pos' in name and 'ner' not in name:
            temp = []
            pos_features = load_mvsa_feature('pos-tfidf')
            temp.append(np.concatenate((textual_features[0], pos_features[0]), axis=1))
            temp.append(np.concatenate((textual_features[1], pos_features[1]), axis=1))
            textual_features = temp

        elif 'pos' not in name and 'ner' in name:
            temp = []
            ner_features = load_mvsa_feature('ner-tfidf')
            temp.append(np.concatenate((textual_features[0], ner_features[0]), axis=1))
            temp.append(np.concatenate((textual_features[1], ner_features[1]), axis=1))
            textual_features = temp

        elif 'pos' in name and 'ner' in name:
            temp = []
            pos_features = load_mvsa_feature('pos-tfidf')
            ner_features = load_mvsa_feature('ner-tfidf')
            temp.append(np.concatenate((textual_features[0], pos_features[0], ner_features[0]), axis=1))
            temp.append(np.concatenate((textual_features[1], pos_features[1], ner_features[1]), axis=1))
            textual_features = temp

        mvsa_single_features.append([textual_features[0], visual_features[0]])
        mvsa_multiple_features.append([textual_features[1], visual_features[1]])

    return mvsa_single_features, mvsa_multiple_features

In [6]:
# e.g. validation_split=0.1 -----> 8:1:1 ratio of train, val, test
def split_data(data, validation_split):
    num_val = int(validation_split * data.shape[0])
    data_train = data[:-(num_val*2)]
    data_val = data[-(num_val*2):-(num_val)]
    data_test = data[-num_val:]
    return data_train, data_val, data_test

In [7]:
def shuffle_mvsa(mvsa_features, labels, indices):
    shuffled_features = []
#     random_idx = np.random.permutation(len(labels))
    for i in range(len(mvsa_features)):
        x, y = mvsa_features[i][0][indices], mvsa_features[i][1][indices]
        shuffled_features.append([x, y])
    return shuffled_features, labels[indices]

def preprocess_inputs(X1, X2, y):
    y = le.fit_transform(y)
    y = to_categorical(np.asarray(y))
    
    X1_train, X1_val, X1_test = split_data(X1, VALIDATION_SPLIT)
    X2_train, X2_val, X2_test = split_data(X2, VALIDATION_SPLIT)
    y_train, y_val, y_test = split_data(y, VALIDATION_SPLIT)

    oversample = BorderlineSMOTE(sampling_strategy='minority', random_state=SEED, kind='borderline-2')
#     oversample = SMOTE(sampling_strategy='minority', random_state=SEED)
    X1_train, _ = oversample.fit_resample(X1_train, y_train)
    X2_train, y_train = oversample.fit_resample(X2_train, y_train)

    return {'texts': [X1_train, X1_val, X1_test], 'images': [X2_train, X2_val, X2_test], 'labels':[y_train, y_val, y_test]}

# def get_preprocess_input(feature_names, mvsa_features, labels):
#     mvsa_features_shuffled, labels_shuffled = shuffle_mvsa(mvsa_features, labels)
#     mvsa_features_split = []
#     for i in range(len(feature_names)):
#         preprocess_splits = preprocess_inputs(mvsa_features_shuffled[i][0], mvsa_features_shuffled[i][1], labels_shuffled)
#         mvsa_features_split.append(preprocess_splits)
#     return mvsa_features_split

def process_dup(names):
    new_names = []
    for i in range(len(names)):
        count_dup = 0
        for j in range(0, i+1):
            if names[i] == names[j]:
                count_dup += 1
        if count_dup > 1:
            new_names.append(names[i] + '-' + str(count_dup))
        else:
            new_names.append(names[i])
    return new_names

In [8]:
def weighted_average(weights, probs):
    ''' Calculate the weighted average probability distribution from all input probs and its weights 
    weights: weights list (or array)
    probs: probability distributions array list
    '''
    output_probs = []
    weighted_probs = [probs[i] * weights[i] for i in range(len(weights))]
    for i in range(len(probs[0])):
        sum_prob = np.zeros(len(probs[0][0]))
        for j in range(len(weights)):
            sum_prob = np.sum((sum_prob, weighted_probs[j][i]), axis=0)
        output_probs.append(sum_prob)
    return np.asarray(output_probs, dtype='float32')

def get_average_weights(*scores, inverse=False):
    ''' Get the corresponding weight of each input score 
    inverse: (bool) get inverse weights value in case of the smaller score value, the bigger weight value (such as model loss)
    '''
    
    weights = []
    for score in scores:
        weights.append(score/np.sum(scores))
    
    if inverse == True:
        inverse_weights = []
        inverse = [1/weight for weight in weights]
        for inv in inverse:
            inverse_weights.append(inv/np.sum(inverse))
        weights = inverse_weights

    return weights

In [9]:
NUM_CLASSES = 3
f1_macro = tfa.metrics.F1Score(num_classes=NUM_CLASSES, average='macro', name='f1_macro')
f1_weighted = tfa.metrics.F1Score(num_classes=NUM_CLASSES, average='weighted', name='f1_weighted')
    
def create_model_text(input_shape, lstm=True):
    text_input = Input(shape=input_shape)
    dropout = Dropout(DROPOUT_INPUT) (text_input)
    if lstm == True:
        text_reshape = Reshape((1, -1)) (dropout)
        text_lstm = LSTM(NUM_LSTM) (text_reshape)
        dropout = Dropout(DROPOUT_LSTM) (text_lstm)
    outputs = Dense(NUM_CLASSES, activation='softmax') (dropout)
    model = Model(text_input, outputs)
    model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=['accuracy', f1_macro, f1_weighted])
    return model

def create_model_image(input_shape, lstm=True):
    image_input = Input(shape=input_shape)
    dropout = Dropout(0.2) (image_input)    
    if lstm == True:
        image_reshape = Reshape((1, -1)) (dropout)
        image_lstm = LSTM(NUM_LSTM_IMG) (image_reshape)
        dropout = Dropout(0.2) (image_lstm)
    outputs = Dense(NUM_CLASSES, activation='softmax') (dropout)
    model = Model(image_input, outputs)
    model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=['accuracy', f1_macro, f1_weighted])
    return model

def create_model_IF(text_shape, image_shape, lstm=True):
    text_input = Input(shape=text_shape)
    image_input = Input(shape=image_shape)
    
    text_dropout = Dropout(DROPOUT_INPUT) (text_input)    
    image_dropout = Dropout(0.2) (image_input)    

    if lstm == True:
        text_reshape = Reshape((1, -1)) (text_dropout)
        text_lstm = LSTM(NUM_LSTM) (text_reshape)
        text_dropout = Dropout(DROPOUT_LSTM) (text_lstm)

        image_reshape = Reshape((1, -1)) (image_dropout)
        image_lstm = LSTM(NUM_LSTM_IMG) (image_reshape)
        image_dropout = Dropout(0.2) (image_lstm)
    
    text_image_concat = tf.keras.layers.Concatenate(axis=1)([text_dropout, image_dropout])
    concat_reshape = Reshape((1, -1)) (text_image_concat)
    self_attention = SeqSelfAttention() (concat_reshape)
#     dropout = Dropout(DROPOUT_ATT) (self_attention)
    flatten = GlobalMaxPooling1D () (self_attention)
    outputs = Dense(NUM_CLASSES, activation='softmax') (flatten)
    model = Model([text_input, image_input], outputs)
    model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=['accuracy', f1_macro, f1_weighted])
    return model

2022-07-20 11:27:04.084844: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [10]:
def run_and_evaluate_IF(name, X1, X2, y, verbose=0, lstm=True):

    data = preprocess_inputs(X1, X2, y)
    X1_train, X1_val, X1_test = data['texts']
    X2_train, X2_val, X2_test = data['images']
    y_train, y_val, y_test = data['labels']
    
    early_stopping_IF = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=EARLY_STOPPING)
    
    if 'multiple' in name:
        batch_size = 256
    else:
        batch_size = BATCH_SIZE # 128

    checkpoint_IF_path = './model_checkpoint/{}.h5'.format(name.split('-')[0] + '-' + '-'.join(name.split('-')[2:]))
    history_IF_path = './model_history/{}.npy'.format(name.split('-')[0] + '-' + '-'.join(name.split('-')[2:]))

    if not(os.path.exists(checkpoint_IF_path) and os.path.exists(history_IF_path)):
        print('Create new IF model:', os.path.split(checkpoint_IF_path)[1])

        model_IF = create_model_IF(X1_train.shape[1:], X2_train.shape[1:], lstm=lstm)
        checkpoint_IF = ModelCheckpoint(checkpoint_IF_path, save_best_only=True, verbose=verbose)
        history_IF = model_IF.fit([X1_train, X2_train], y_train, validation_data=([X1_val, X2_val], y_val), 
                            epochs=EPOCHS, batch_size=batch_size, verbose=verbose,
                            callbacks=[checkpoint_IF, early_stopping_IF])
        if not os.path.exists(os.path.split(history_IF_path)[0]):
            os.makedirs(os.path.split(history_IF_path)[0])
        pickle.dump(history_IF.history, open(history_IF_path, 'wb'))
        
    model_IF = load_model(checkpoint_IF_path, custom_objects={'SeqSelfAttention': SeqSelfAttention})
    history_IF = pickle.load(open(history_IF_path, 'rb'))
    
    best_epoch = np.argmin(history_IF['val_loss'])
    print('Model IF checkpoint loaded at epoch:', best_epoch)

    return history_IF, evaluate_model_IF(model_IF, X1_test, X2_test, y_test, verbose=verbose)

In [11]:
def run_and_evaluate_LF(name, X1, X2, y, verbose=0, lstm=True):

    data = preprocess_inputs(X1, X2, y)
    X1_train, X1_val, X1_test = data['texts']
    X2_train, X2_val, X2_test = data['images']
    y_train, y_val, y_test = data['labels']

    early_stopping_text = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=EARLY_STOPPING)
    early_stopping_image = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=EARLY_STOPPING)

    if 'multiple' in name:
        batch_size = 256
    else:
        batch_size = BATCH_SIZE # 128

    checkpoint_text_path = './model_checkpoint/{}.h5'.format(name.split('-')[0] + '-' + '-'.join(name.split('-')[3:]))
    checkpoint_image_path = './model_checkpoint/{}.h5'.format(name.split('-')[0] + '-' + name.split('-')[2])

    history_text_path = './model_history/{}.npy'.format(name.split('-')[0] + '-' + '-'.join(name.split('-')[3:]))
    history_image_path = './model_history/{}.npy'.format(name.split('-')[0] + '-' + name.split('-')[2])

    if lstm == True:
        checkpoint_image_path = './model_checkpoint/{}.h5'.format(name.split('-')[0] + '-' + name.split('-')[2] + '-lstm')
        history_image_path = './model_history/{}.npy'.format(name.split('-')[0] + '-' + name.split('-')[2] + '-lstm')
    
    if not (os.path.exists(checkpoint_text_path) and os.path.exists(history_text_path)):
        print('Create new text model:', os.path.split(checkpoint_text_path)[1])

        model_text = create_model_text(X1_train.shape[1:], lstm=lstm)
        checkpoint_text = ModelCheckpoint(checkpoint_text_path, save_best_only=True, verbose=verbose)
        history_text = model_text.fit(X1_train, y_train, validation_data=(X1_val, y_val), 
                                  epochs=EPOCHS, batch_size=batch_size, verbose=verbose,
                                  callbacks=[checkpoint_text, early_stopping_text])
        if not os.path.exists(os.path.split(history_text_path)[0]):
            os.makedirs(os.path.split(history_text_path)[0])
        pickle.dump(history_text.history, open(history_text_path, 'wb'))
    
    if not(os.path.exists(checkpoint_image_path) and os.path.exists(history_image_path)):
        print('Create new image model:', os.path.split(checkpoint_image_path)[1])

        model_image = create_model_image(X2_train.shape[1:], lstm=lstm)
        checkpoint_image = ModelCheckpoint(checkpoint_image_path, 
                                       save_best_only=True, verbose=verbose)
        history_image = model_image.fit(X2_train, y_train, validation_data=(X2_val, y_val), 
                                epochs=EPOCHS, batch_size=batch_size, verbose=verbose,
                                callbacks=[checkpoint_image, early_stopping_image])
        pickle.dump(history_image.history, open(history_image_path, 'wb'))
    
    model_text = load_model(checkpoint_text_path)
    model_image = load_model(checkpoint_image_path)

    history_text = pickle.load(open(history_text_path, 'rb'))    
    history_image = pickle.load(open(history_image_path, 'rb'))

    y_pred_text = model_text.predict(X1_test)
    y_pred_image = model_image.predict(X2_test)

    best_epoch_text = np.argmin(history_text['val_loss'])
    best_epoch_image = np.argmin(history_image['val_loss'])
    print('Model text checkpoint loaded at epoch:', best_epoch_text)
    print('Model image checkpoint loaded at epoch:', best_epoch_image)
    
    val_acc_text = history_text['val_accuracy'][best_epoch_text]
    val_acc_image = history_image['val_accuracy'][best_epoch_image]
    weights = get_average_weights(val_acc_text, val_acc_image)
    print('Model weights (text, image):', weights)

    y_pred = weighted_average(weights, np.asarray([y_pred_text, y_pred_image], dtype='float32'))
    
    eval_text = evaluate_model_uni(model_text, X1_test, y_test, verbose=verbose)
    eval_image = evaluate_model_uni(model_image, X2_test, y_test, verbose=verbose)
    eval_LF = evaluate_model_LF(y_test, y_pred, verbose=verbose)
    return eval_text, eval_image, eval_LF

In [12]:
def run_and_evaluate_HF(name, X1, X2, y, verbose=0, lstm=True):
    
    data = preprocess_inputs(X1, X2, y)
    X1_train, X1_val, X1_test = data['texts']
    X2_train, X2_val, X2_test = data['images']
    y_train, y_val, y_test = data['labels']

    early_stopping_text = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=EARLY_STOPPING)
    early_stopping_image = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=EARLY_STOPPING)
    early_stopping_IF = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=EARLY_STOPPING)

    if 'multiple' in name:
        batch_size = 256
    else:
        batch_size = BATCH_SIZE # 128
        
    checkpoint_text_path = './model_checkpoint/{}.h5'.format(name.split('-')[0] + '-' + '-'.join(name.split('-')[3:]))
    checkpoint_image_path = './model_checkpoint/{}.h5'.format(name.split('-')[0] + '-' + name.split('-')[2])
    checkpoint_IF_path = './model_checkpoint/{}.h5'.format(name.split('-')[0] + '-' + '-'.join(name.split('-')[2:]))

    history_text_path = './model_history/{}.npy'.format(name.split('-')[0] + '-' + '-'.join(name.split('-')[3:]))
    history_image_path = './model_history/{}.npy'.format(name.split('-')[0] + '-' + name.split('-')[2])
    history_IF_path = './model_history/{}.npy'.format(name.split('-')[0] + '-' + '-'.join(name.split('-')[2:]))
    
    if lstm == True:
        checkpoint_image_path = './model_checkpoint/{}.h5'.format(name.split('-')[0] + '-' + name.split('-')[2] + '-lstm')
        history_image_path = './model_history/{}.npy'.format(name.split('-')[0] + '-' + name.split('-')[2] + '-lstm')

    if not(os.path.exists(checkpoint_text_path) and os.path.exists(history_text_path)):
        print('Create new text model:', os.path.split(checkpoint_text_path)[1])

        model_text = create_model_text(X1_train.shape[1:], lstm=lstm)
        checkpoint_text = ModelCheckpoint(checkpoint_text_path, save_best_only=True, verbose=verbose)
        history_text = model_text.fit(X1_train, y_train, validation_data=(X1_val, y_val), 
                                  epochs=EPOCHS, batch_size=batch_size, verbose=verbose,
                                  callbacks=[checkpoint_text, early_stopping_text])
        if not os.path.exists(os.path.split(history_text_path)[0]):
            os.makedirs(os.path.split(history_text_path)[0])
        pickle.dump(history_text.history, open(history_text_path, 'wb'))
    
    if not(os.path.exists(checkpoint_image_path) and os.path.exists(history_image_path)):
        print('Create new image model:', os.path.split(checkpoint_image_path)[1])

        model_image = create_model_image(X2_train.shape[1:], lstm=lstm)        
        checkpoint_image = ModelCheckpoint(checkpoint_image_path, save_best_only=True, verbose=verbose)
        history_image = model_image.fit(X2_train, y_train, validation_data=(X2_val, y_val), 
                                epochs=EPOCHS, batch_size=batch_size, verbose=verbose,
                                callbacks=[checkpoint_image, early_stopping_image])
        pickle.dump(history_image.history, open(history_image_path, 'wb'))
    
    if not(os.path.exists(checkpoint_IF_path) and os.path.exists(history_IF_path)):
        print('Create new IF model:', os.path.split(checkpoint_IF_path)[1])

        model_IF = create_model_IF(X1_train.shape[1:], X2_train.shape[1:], lstm=lstm)
        checkpoint_IF = ModelCheckpoint(checkpoint_IF_path, save_best_only=True, verbose=verbose)
        history_IF = model_IF.fit([X1_train, X2_train], y_train, validation_data=([X1_val, X2_val], y_val), 
                            epochs=EPOCHS, batch_size=batch_size, verbose=verbose,
                            callbacks=[checkpoint_IF, early_stopping_IF])
        pickle.dump(history_IF.history, open(history_IF_path, 'wb'))

    model_text = load_model(checkpoint_text_path)
    model_image = load_model(checkpoint_image_path)
    model_IF = load_model(checkpoint_IF_path, custom_objects={'SeqSelfAttention': SeqSelfAttention})

    history_image = pickle.load(open(history_image_path, 'rb'))
    history_text = pickle.load(open(history_text_path, 'rb'))
    history_IF = pickle.load(open(history_IF_path, 'rb'))

    y_pred_text = model_text.predict(X1_test)
    y_pred_image = model_image.predict(X2_test)
    y_pred_IF = model_IF.predict([X1_test, X2_test])
    
    best_epoch_text = np.argmin(history_text['val_loss'])
    best_epoch_image = np.argmin(history_image['val_loss'])
    best_epoch_IF = np.argmin(history_IF['val_loss'])
    print('Model text checkpoint loaded at epoch:', best_epoch_text)
    print('Model image checkpoint loaded at epoch:', best_epoch_image)
    print('Model IF checkpoint loaded at epoch:', best_epoch_IF)

    val_acc_text = history_text['val_accuracy'][best_epoch_text]
    val_acc_image = history_image['val_accuracy'][best_epoch_image]
    val_acc_IF = history_IF['val_accuracy'][best_epoch_IF]
    
    weights = get_average_weights(val_acc_text, val_acc_image, val_acc_IF)
    print('Model weights (text, image, IF):', weights)
    y_pred = weighted_average(weights, np.asarray([y_pred_text, y_pred_image, y_pred_IF], dtype='float32'))

    eval_text = evaluate_model_uni(model_text, X1_test, y_test, verbose=verbose)
    eval_image = evaluate_model_uni(model_image, X2_test, y_test, verbose=verbose)
    eval_IF = evaluate_model_IF(model_IF, X1_test, X2_test, y_test, verbose=verbose)
    eval_HF = evaluate_model_LF(y_test, y_pred, verbose=verbose)
    return eval_text, eval_image, eval_IF, eval_HF

In [13]:
def evaluate_model_IF(model, X_texts, X_images, y_test, verbose=1):

    loss, acc, f1_macro, f1_weighted = model.evaluate([X_texts, X_images], y_test, verbose=verbose)

    if verbose == 1:
        print('Loss:', loss)
        print('Accuracy:', acc)
        print('Macro F1-score:', f1_macro)
        print('Weighted F1-score:', f1_weighted)

        y_pred = model.predict([X_texts, X_images])
        matrix = confusion_matrix(le.inverse_transform(y_test.argmax(axis=1)), le.inverse_transform(y_pred.argmax(axis=1)), 
                                  labels=list(le.classes_))
        cm_disp = ConfusionMatrixDisplay(confusion_matrix=matrix,
                                  display_labels=list(le.classes_))
        cm_disp.plot()
        plt.show()
        
    return acc, f1_macro, f1_weighted

def evaluate_model_LF(y_true, y_pred, verbose=0):
    
    y_pred = le.inverse_transform(y_pred.argmax(axis=1))
    y_true = le.inverse_transform(y_true.argmax(axis=1))

    acc = accuracy_score(y_true, y_pred)
    f1_macro = f1_score(y_true, y_pred, average='macro')
    f1_weighted = f1_score(y_true, y_pred, average='weighted')

    if verbose == 1:
        print(classification_report(y_true, y_pred))
        matrix = confusion_matrix(y_true, y_pred,
                                  labels=list(le.classes_))
        cm_disp = ConfusionMatrixDisplay(confusion_matrix=matrix,
                                  display_labels=list(le.classes_))
        cm_disp.plot()
        plt.show()

    return acc, f1_macro, f1_weighted

def evaluate_model_uni(model, X_test, y_test, verbose=1):
    
    loss, acc, f1_macro, f1_weighted = model.evaluate(X_test, y_test, verbose=verbose)
    
    if verbose == 1:
        print('Loss:', loss)
        print('Accuracy:', acc)
        print('Macro F1-score:', f1_macro)
        print('Weighted F1-score:', f1_weighted)
        
    return acc, f1_macro, f1_weighted

In [14]:
def plot_metrics(history):
    fig = plt.figure(figsize=(20, 5))

    fig.add_subplot(1, 4, 1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('LOSS')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')

    fig.add_subplot(1, 4, 2)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('ACCURACY')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')

    fig.add_subplot(1, 4, 3)
    plt.plot(history.history['f1_macro'])
    plt.plot(history.history['val_f1_macro'])
    plt.title('Macro F1-SCORE')
    plt.ylabel('f1-macro')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')
    
    fig.add_subplot(1, 4, 4)
    plt.plot(history.history['f1_weighted'])
    plt.plot(history.history['val_f1_weighted'])
    plt.title('Weighted F1-SCORE')
    plt.ylabel('f1-weighted')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='best')

    plt.show()

In [15]:
def style_dataframe(dataframe):
    return dataframe.style.highlight_max(subset=['Accuracy', 'F1-macro', 'F1-weighted'], props='color:lawngreen', axis=0)\
                          .highlight_min(subset=['Accuracy', 'F1-macro', 'F1-weighted'], props='color:tomato', axis=0)

def highlight_neg(cell):
    if type(cell) != str and cell < 0 :
        return 'color: tomato'
    else:
        return 'color: lawngreen'

def display_dataframes(dfs, names=[], index=False):
    def to_df(x):
        if isinstance(x, pd.Series):
            return pd.DataFrame(x)
        else:
            return x
    html_str = ''
    if names:
        html_str += ('<tr>' + 
                     ''.join(f'<td style="text-align:center">{name}</td>' for name in names) + 
                     '</tr>')
    html_str += ('<tr>' + 
                 ''.join(f'<td style="vertical-align:top"> {to_df(df).to_html()}</td>' 
                         for df in dfs) + 
                 '</tr>')
    html_str = f'<table>{html_str}</table>'
    html_str = html_str.replace('table','table style="display:inline"')
    display_html(html_str, raw=True)

# Load data

In [16]:
mvsa_single_multimodal_labels, _, _ = load_labels('../input/mvsa-features/labels/mvsa-single-labels.hdf5')
mvsa_multiple_multimodal_labels, _, _ = load_labels('../input/mvsa-features/labels/mvsa-multiple-labels.hdf5')

le = LabelEncoder()
le.fit(mvsa_multiple_multimodal_labels)
NUM_CLASSES = len(le.classes_) # = 3
mapping = dict(zip(range(len(le.classes_)), le.classes_))
print(mapping)

{0: 'negative', 1: 'neutral', 2: 'positive'}


In [17]:
feature_names = ['resnet101-bert', 'resnet101-bert-lstm', 'densenet201-bert-lstm', 'densenet201-bert-pos-lstm', 'densenet201-bert-ner-lstm', 'densenet201-bert-pos-ner-lstm']
visual_feature_names = process_dup([name.split('-')[0] for name in feature_names])
textual_feature_names = process_dup(['-'.join(name.split('-')[1:]) for name in feature_names])
# for i in range(len(feature_names)):
#     x = ' '.join(feature_names[i].split('-')[1:]).rstrip()
#     if len(x.split()) > 1:
#         x = '-'.join(x.rstrip('-lstm').rstrip().split())
#     textual_feature_names.append(x)

In [18]:
mvsa_single_features, mvsa_multiple_features = get_features(feature_names)

In [19]:
# Fix random indices for consistency between other experiments
mvsa_single_features, mvsa_single_multimodal_labels = shuffle_mvsa(mvsa_single_features, mvsa_single_multimodal_labels, np.load('../input/mvsa-shuffle-indices/mvsa-single-shuffle-indices.npy'))
mvsa_multiple_features, mvsa_multiple_multimodal_labels = shuffle_mvsa(mvsa_multiple_features, mvsa_multiple_multimodal_labels, np.load('../input/mvsa-shuffle-indices/mvsa-multiple-shuffle-indices.npy'))

In [20]:
reset_seeds()
EPOCHS = 100
BATCH_SIZE = 128
VALIDATION_SPLIT = 0.1
EARLY_STOPPING = 10
NUM_LSTM_IMG = 128
NUM_LSTM = 128
DROPOUT_INPUT = 0.4
DROPOUT_LSTM = 0.5
DROPOUT_ATT = 0.0
OPTIMIZER = 'adam'
LOSS = 'categorical_crossentropy'

In [21]:
# mvsa_single_features_split = get_preprocess_input(feature_names, mvsa_single_features, mvsa_single_multimodal_labels)
# mvsa_multiple_features_split = get_preprocess_input(feature_names, mvsa_multiple_features, mvsa_multiple_multimodal_labels)
# mvsa_single_features, mvsa_single_multimodal_labels = shuffle_mvsa(mvsa_single_features, mvsa_single_multimodal_labels)
# mvsa_multiple_features, mvsa_multiple_multimodal_labels = shuffle_mvsa(mvsa_multiple_features, mvsa_multiple_multimodal_labels)

# Intermediate Fusion

In [22]:
print('MVSA-Single with Intermediate Fusion')
scores = []
for i in range(len(feature_names)):
    print('MVSA-Single:', feature_names[i])
    if 'lstm' in feature_names[i]:
        _, score = run_and_evaluate_IF('single-IF-' + feature_names[i], 
                                       mvsa_single_features[i][0],
                                       mvsa_single_features[i][1],
                                       mvsa_single_multimodal_labels, 
                                       verbose=0)
    else:
        _, score = run_and_evaluate_IF('single-IF-' + feature_names[i], 
                                       mvsa_single_features[i][0],
                                       mvsa_single_features[i][1],
                                       mvsa_single_multimodal_labels, 
                                       verbose=0, lstm=False)
    scores.append(score)
    print()
df0_single_scores_IF = pd.DataFrame(scores, columns=['Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

print('----------------------------------------')
print('\nMVSA-Multiple with Intermediate Fusion')
scores = []
for i in range(len(feature_names)):
    print('MVSA-Multiple:', feature_names[i])
    if 'lstm' in feature_names[i]:
        _, score = run_and_evaluate_IF('multiple-IF-' + feature_names[i], 
                                       mvsa_multiple_features[i][0],
                                       mvsa_multiple_features[i][1],
                                       mvsa_multiple_multimodal_labels, 
                                       verbose=0)
    else:
        _, score = run_and_evaluate_IF('multiple-IF-' + feature_names[i], 
                                       mvsa_multiple_features[i][0],
                                       mvsa_multiple_features[i][1],
                                       mvsa_multiple_multimodal_labels, 
                                       verbose=0, lstm=False)
    scores.append(score)
    print()
df0_multiple_scores_IF = pd.DataFrame(scores, columns=['Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

MVSA-Single with Intermediate Fusion
MVSA-Single: resnet101-bert
Create new IF model: single-resnet101-bert.h5


2022-07-20 11:27:13.431972: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Model IF checkpoint loaded at epoch: 27

MVSA-Single: resnet101-bert-lstm
Create new IF model: single-resnet101-bert-lstm.h5
Model IF checkpoint loaded at epoch: 22

MVSA-Single: densenet201-bert-lstm
Create new IF model: single-densenet201-bert-lstm.h5
Model IF checkpoint loaded at epoch: 4

MVSA-Single: densenet201-bert-pos-lstm
Create new IF model: single-densenet201-bert-pos-lstm.h5
Model IF checkpoint loaded at epoch: 4

MVSA-Single: densenet201-bert-ner-lstm
Create new IF model: single-densenet201-bert-ner-lstm.h5
Model IF checkpoint loaded at epoch: 4

MVSA-Single: densenet201-bert-pos-ner-lstm
Create new IF model: single-densenet201-bert-pos-ner-lstm.h5
Model IF checkpoint loaded at epoch: 4

----------------------------------------

MVSA-Multiple with Intermediate Fusion
MVSA-Multiple: resnet101-bert
Create new IF model: multiple-resnet101-bert.h5
Model IF checkpoint loaded at epoch: 17

MVSA-Multiple: resnet101-bert-lstm
Create new IF model: multiple-resnet101-bert-lstm.h5
Mo

# Late Fusion

In [23]:
print('MVSA-Single with Late Fusion')
scores_text = []
scores_image = []
scores_LF = []
for i in range(len(feature_names)):
    print('MVSA-Single:', feature_names[i])
    if 'lstm' in feature_names[i]:
        score_text, score_image, score_LF = run_and_evaluate_LF('single-LF-' + feature_names[i],
                                                                mvsa_single_features[i][0], 
                                                                mvsa_single_features[i][1],
                                                                mvsa_single_multimodal_labels, 
                                                                verbose=0)
    else:
        score_text, score_image, score_LF = run_and_evaluate_LF('single-LF-' + feature_names[i],
                                                                mvsa_single_features[i][0], 
                                                                mvsa_single_features[i][1],
                                                                mvsa_single_multimodal_labels, 
                                                                verbose=0, lstm=False)    
    scores_text.append(score_text)
    scores_image.append(score_image)
    scores_LF.append(score_LF)
    print()

df1_single_scores_text = pd.DataFrame(scores_text, columns=['Accuracy', 'F1-macro', 'F1-weighted'], index=textual_feature_names)
df1_single_scores_image = pd.DataFrame(scores_image, columns=['Accuracy', 'F1-macro', 'F1-weighted'], index=visual_feature_names)
df1_single_scores_LF = pd.DataFrame(scores_LF, columns=['Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

print('--------------------------------')
print('\nMVSA-Multiple with Late Fusion')
scores_text = []
scores_image = []
scores_LF = []
for i in range(len(feature_names)):
    print('MVSA-Multiple:', feature_names[i])
    if 'lstm' in feature_names[i]:
        score_text, score_image, score_LF = run_and_evaluate_LF('multiple-LF-' + feature_names[i],
                                                                mvsa_multiple_features[i][0], 
                                                                mvsa_multiple_features[i][1],
                                                                mvsa_multiple_multimodal_labels, 
                                                                verbose=0)
    else:
        score_text, score_image, score_LF = run_and_evaluate_LF('multiple-LF-' + feature_names[i],
                                                                mvsa_multiple_features[i][0], 
                                                                mvsa_multiple_features[i][1],
                                                                mvsa_multiple_multimodal_labels, 
                                                                verbose=0, lstm=False)    
    scores_text.append(score_text)
    scores_image.append(score_image)
    scores_LF.append(score_LF)
    print()

df1_multiple_scores_text = pd.DataFrame(scores_text, columns=['Accuracy', 'F1-macro', 'F1-weighted'], index=textual_feature_names)
df1_multiple_scores_image = pd.DataFrame(scores_image, columns=['Accuracy', 'F1-macro', 'F1-weighted'], index=visual_feature_names)
df1_multiple_scores_LF = pd.DataFrame(scores_LF, columns=['Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

MVSA-Single with Late Fusion
MVSA-Single: resnet101-bert
Create new text model: single-bert.h5
Create new image model: single-resnet101.h5
Model text checkpoint loaded at epoch: 15
Model image checkpoint loaded at epoch: 12
Model weights (text, image): [0.5365853591905375, 0.4634146408094625]

MVSA-Single: resnet101-bert-lstm
Create new text model: single-bert-lstm.h5
Create new image model: single-resnet101-lstm.h5
Model text checkpoint loaded at epoch: 15
Model image checkpoint loaded at epoch: 22
Model weights (text, image): [0.5507518840683405, 0.4492481159316595]

MVSA-Single: densenet201-bert-lstm
Create new image model: single-densenet201-lstm.h5
Model text checkpoint loaded at epoch: 15
Model image checkpoint loaded at epoch: 5
Model weights (text, image): [0.5213522927406988, 0.4786477072593012]

MVSA-Single: densenet201-bert-pos-lstm
Create new text model: single-bert-pos-lstm.h5
Model text checkpoint loaded at epoch: 15
Model image checkpoint loaded at epoch: 5
Model weights

# Hybrid Fusion

In [24]:
print('MVSA-Single with Hybrid Fusion')
scores_text = []
scores_image = []
scores_IF = []
scores_HF = []
for i in range(len(feature_names)):
    print('MVSA-Single:', feature_names[i])
    if 'lstm' in feature_names[i]:
        score_text, score_image, score_IF, score_HF = run_and_evaluate_HF('single-HF-' + feature_names[i], 
                                                                          mvsa_single_features[i][0], 
                                                                          mvsa_single_features[i][1], 
                                                                          mvsa_single_multimodal_labels, 
                                                                          verbose=0)
    else:
        score_text, score_image, score_IF, score_HF = run_and_evaluate_HF('single-HF-' + feature_names[i], 
                                                                          mvsa_single_features[i][0], 
                                                                          mvsa_single_features[i][1], 
                                                                          mvsa_single_multimodal_labels, 
                                                                          verbose=0, lstm=False)
    scores_text.append(score_text)
    scores_image.append(score_image)
    scores_IF.append(score_IF)
    scores_HF.append(score_HF)
    print()

df2_single_scores_text = pd.DataFrame(scores_text, columns=['Accuracy', 'F1-macro', 'F1-weighted'], index=textual_feature_names)
df2_single_scores_image = pd.DataFrame(scores_image, columns=['Accuracy', 'F1-macro', 'F1-weighted'], index=visual_feature_names)
df2_single_scores_IF = pd.DataFrame(scores_IF, columns=['Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)
df2_single_scores_HF = pd.DataFrame(scores_HF, columns=['Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

print('----------------------------------')
print('\nMVSA-Multiple with Hybrid Fusion')
scores_text = []
scores_image = []
scores_IF = []
scores_HF = []
for i in range(len(feature_names)):
    print('MVSA-Multiple:', feature_names[i])
    if 'lstm' in feature_names[i]:
        score_text, score_image, score_IF, score_HF = run_and_evaluate_HF('multiple-HF-' + feature_names[i], 
                                                                          mvsa_multiple_features[i][0], 
                                                                          mvsa_multiple_features[i][1], 
                                                                          mvsa_multiple_multimodal_labels, 
                                                                          verbose=0)
    else:
        score_text, score_image, score_IF, score_HF = run_and_evaluate_HF('multiple-HF-' + feature_names[i], 
                                                                          mvsa_multiple_features[i][0], 
                                                                          mvsa_multiple_features[i][1], 
                                                                          mvsa_multiple_multimodal_labels, 
                                                                          verbose=0, lstm=False)
    scores_text.append(score_text)
    scores_image.append(score_image)
    scores_IF.append(score_IF)
    scores_HF.append(score_HF)
    print()

df2_multiple_scores_text = pd.DataFrame(scores_text, columns=['Accuracy', 'F1-macro', 'F1-weighted'], index=textual_feature_names)
df2_multiple_scores_image = pd.DataFrame(scores_image, columns=['Accuracy', 'F1-macro', 'F1-weighted'], index=visual_feature_names)
df2_multiple_scores_IF = pd.DataFrame(scores_IF, columns=['Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)
df2_multiple_scores_HF = pd.DataFrame(scores_HF, columns=['Accuracy', 'F1-macro', 'F1-weighted'], index=feature_names)

MVSA-Single with Hybrid Fusion
MVSA-Single: resnet101-bert
Model text checkpoint loaded at epoch: 15
Model image checkpoint loaded at epoch: 12
Model IF checkpoint loaded at epoch: 27
Model weights (text, image, IF): [0.3441981644253396, 0.29726205906003017, 0.3585397765146302]

MVSA-Single: resnet101-bert-lstm
Model text checkpoint loaded at epoch: 15
Model image checkpoint loaded at epoch: 22
Model IF checkpoint loaded at epoch: 22
Model weights (text, image, IF): [0.35601458183382684, 0.2904009677671809, 0.3535844503989923]

MVSA-Single: densenet201-bert-lstm
Model text checkpoint loaded at epoch: 15
Model image checkpoint loaded at epoch: 5
Model IF checkpoint loaded at epoch: 4
Model weights (text, image, IF): [0.33794692871368365, 0.31026529442075235, 0.351787776865564]

MVSA-Single: densenet201-bert-pos-lstm
Model text checkpoint loaded at epoch: 15
Model image checkpoint loaded at epoch: 5
Model IF checkpoint loaded at epoch: 4
Model weights (text, image, IF): [0.33644858463012

# Display results

In [25]:
print('Intermediate Fusion')
display_dataframes((style_dataframe(df0_single_scores_IF), style_dataframe(df0_multiple_scores_IF)), 
                   names=['MVSA-Single', 'MVSA-Multiple'])

Intermediate Fusion


MVSA-Single MVSA-Multiple 
 
 
 
   
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 resnet101-bert 
 0.656319 
 0.541512 
 0.656621 
 
 
 resnet101-bert-lstm 
 0.665188 
 0.546898 
 0.666523 
 
 
 densenet201-bert-lstm 
 0.674058 
 0.575673 
 0.673735 
 
 
 densenet201-bert-pos-lstm 
 0.669623 
 0.555386 
 0.665440 
 
 
 densenet201-bert-ner-lstm 
 0.667406 
 0.570923 
 0.672192 
 
 
 densenet201-bert-pos-ner-lstm 
 0.660754 
 0.544587 
 0.651359 
 
 
 
 
 
 
 
   
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 resnet101-bert 
 0.642186 
 0.469748 
 0.616122 
 
 
 resnet101-bert-lstm 
 0.659224 
 0.459090 
 0.619891 
 
 
 densenet201-bert-lstm 
 0.675676 
 0.475296 
 0.642300 
 
 
 densenet201-bert-pos-lstm 
 0.669213 
 0.467579 
 0.635927 
 
 
 densenet201-bert-ner-lstm 
 0.675676 
 0.469268 
 0.636860 
 
 
 densenet201-bert-pos-ner-lstm 
 0.681551 
 0.474050 
 0.639784

In [26]:
print('LATE FUSION \n')
print('MVSA-Single')
print(display_dataframes((style_dataframe(df1_single_scores_text), style_dataframe(df1_single_scores_image), 
                          style_dataframe(df1_single_scores_LF)), 
                         names=['Model Text', 'Model Image', 'Model LF']))
print('\nMVSA-Multiple')
print(display_dataframes((style_dataframe(df1_multiple_scores_text), style_dataframe(df1_multiple_scores_image), style_dataframe(df1_multiple_scores_LF)), 
                   names=['Model Text', 'Model Image', 'Model LF']))

LATE FUSION 

MVSA-Single


Model Text Model Image Model LF 
 
 
 
   
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 bert 
 0.574279 
 0.477230 
 0.586566 
 
 
 bert-lstm 
 0.587583 
 0.490084 
 0.596213 
 
 
 bert-lstm-2 
 0.587583 
 0.490084 
 0.596213 
 
 
 bert-pos-lstm 
 0.625277 
 0.516229 
 0.635317 
 
 
 bert-ner-lstm 
 0.611973 
 0.508985 
 0.618135 
 
 
 bert-pos-ner-lstm 
 0.598670 
 0.506415 
 0.614556 
 
 
 
 
 
 
 
   
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 resnet101 
 0.470067 
 0.273038 
 0.405977 
 
 
 resnet101-2 
 0.521064 
 0.291616 
 0.430390 
 
 
 densenet201 
 0.636364 
 0.515845 
 0.619809 
 
 
 densenet201-2 
 0.636364 
 0.515845 
 0.619809 
 
 
 densenet201-3 
 0.636364 
 0.515845 
 0.619809 
 
 
 densenet201-4 
 0.636364 
 0.515845 
 0.619809 
 
 
 
 
 
 
 
   
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 resnet101-bert 
 0.592018 
 0.459830 
 0.569450 
 
 
 resnet101-bert-lstm 
 0.634146 
 0.503554 
 0.617071 
 
 
 densenet201-bert-lstm 
 0.674058 
 0.553236 
 0.661270 
 
 
 densenet201-bert-pos-lstm 
 0.687361 
 0.565339 
 0.673386 
 
 
 densenet201-bert-ner-lstm 
 0.678492 
 0.562090 
 0.663514 
 
 
 densenet201-bert-pos-ner-lstm 
 0.676275 
 0.558625 
 0.663824

None

MVSA-Multiple


Model Text Model Image Model LF 
 
 
 
   
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 bert 
 0.622797 
 0.453437 
 0.603989 
 
 
 bert-lstm 
 0.658049 
 0.461529 
 0.618625 
 
 
 bert-lstm-2 
 0.658049 
 0.461529 
 0.618625 
 
 
 bert-pos-lstm 
 0.658637 
 0.442365 
 0.610535 
 
 
 bert-ner-lstm 
 0.664512 
 0.461616 
 0.621235 
 
 
 bert-pos-ner-lstm 
 0.655699 
 0.449619 
 0.614270 
 
 
 
 
 
 
 
   
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 resnet101 
 0.555817 
 0.333508 
 0.515907 
 
 
 resnet101-2 
 0.618096 
 0.334073 
 0.539235 
 
 
 densenet201 
 0.640423 
 0.393927 
 0.583949 
 
 
 densenet201-2 
 0.640423 
 0.393927 
 0.583949 
 
 
 densenet201-3 
 0.640423 
 0.393927 
 0.583949 
 
 
 densenet201-4 
 0.640423 
 0.393927 
 0.583949 
 
 
 
 
 
 
 
   
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 resnet101-bert 
 0.633373 
 0.418999 
 0.581512 
 
 
 resnet101-bert-lstm 
 0.653937 
 0.407703 
 0.577312 
 
 
 densenet201-bert-lstm 
 0.672738 
 0.441986 
 0.607051 
 
 
 densenet201-bert-pos-lstm 
 0.667450 
 0.428068 
 0.597053 
 
 
 densenet201-bert-ner-lstm 
 0.672150 
 0.438731 
 0.604032 
 
 
 densenet201-bert-pos-ner-lstm 
 0.663925 
 0.428869 
 0.597946

None


In [27]:
print('HYBRID FUSION \n')
print('MVSA-Single')
print(display_dataframes((style_dataframe(df2_single_scores_text), style_dataframe(df2_single_scores_image), 
                          style_dataframe(df2_single_scores_IF), style_dataframe(df2_single_scores_HF)), 
                   names=['Model Text', 'Model Image', 'Model IF', 'Model HF']))
print('\nMVSA-Multiple')
print(display_dataframes((style_dataframe(df2_multiple_scores_text), style_dataframe(df2_multiple_scores_image), 
                          style_dataframe(df2_multiple_scores_IF), style_dataframe(df2_multiple_scores_HF)), 
                   names=['Model Text', 'Model Image', 'Model IF', 'Model HF']))

HYBRID FUSION 

MVSA-Single


Model Text,Model Image,Model IF,Model HF
,Accuracy,F1-macro,F1-weighted
bert,0.574279,0.477230,0.586566
bert-lstm,0.587583,0.490084,0.596213
bert-lstm-2,0.587583,0.490084,0.596213
bert-pos-lstm,0.625277,0.516229,0.635317
bert-ner-lstm,0.611973,0.508985,0.618135
bert-pos-ner-lstm,0.598670,0.506415,0.614556
,Accuracy,F1-macro,F1-weighted
resnet101,0.470067,0.273038,0.405977
resnet101-2,0.521064,0.291616,0.430390


None

MVSA-Multiple


Model Text,Model Image,Model IF,Model HF
,Accuracy,F1-macro,F1-weighted
bert,0.622797,0.453437,0.603989
bert-lstm,0.658049,0.461529,0.618625
bert-lstm-2,0.658049,0.461529,0.618625
bert-pos-lstm,0.658637,0.442365,0.610535
bert-ner-lstm,0.664512,0.461616,0.621235
bert-pos-ner-lstm,0.655699,0.449619,0.614270
,Accuracy,F1-macro,F1-weighted
resnet101,0.555817,0.333508,0.515907
resnet101-2,0.618096,0.334073,0.539235


None


In [28]:
# get max values of late and intermediate fusion
df_single_higher_fusion = pd.DataFrame(np.where(df1_single_scores_LF.gt(df0_single_scores_IF.values), 
                                                df1_single_scores_LF.values, df0_single_scores_IF.values),
                                       columns=['Accuracy','F1-macro','F1-weighted'], index=feature_names)

df_multiple_higher_fusion = pd.DataFrame(np.where(df1_multiple_scores_LF.gt(df0_multiple_scores_IF.values),
                                                  df1_multiple_scores_LF.values, df0_multiple_scores_IF.values),
                                         columns=['Accuracy','F1-macro','F1-weighted'], index=feature_names)

df_single_subtract = df2_single_scores_HF.subtract(df_single_higher_fusion)
df_multiple_subtract = df2_multiple_scores_HF.subtract(df_multiple_higher_fusion)

In [29]:
print('Compare Hybrid Fusion with other Fusion Models (>scores)')
display_dataframes((df_single_subtract.style.applymap(highlight_neg), df_multiple_subtract.style.applymap(highlight_neg)), 
                   names=['MVSA-Single', 'MVSA-Multiple'])

Compare Hybrid Fusion with other Fusion Models (>scores)


MVSA-Single MVSA-Multiple 
 
 
 
   
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 resnet101-bert 
 -0.028825 
 -0.031267 
 -0.036791 
 
 
 resnet101-bert-lstm 
 0.002217 
 0.005357 
 -0.007413 
 
 
 densenet201-bert-lstm 
 0.008869 
 -0.007564 
 0.002298 
 
 
 densenet201-bert-pos-lstm 
 -0.011086 
 -0.002422 
 -0.004225 
 
 
 densenet201-bert-ner-lstm 
 0.008869 
 0.015440 
 0.008874 
 
 
 densenet201-bert-pos-ner-lstm 
 -0.002217 
 0.002442 
 0.002463 
 
 
 
 
 
 
 
   
 Accuracy 
 F1-macro 
 F1-weighted 
 
 
 
 
 resnet101-bert 
 -0.005875 
 -0.032040 
 -0.021607 
 
 
 resnet101-bert-lstm 
 0.004113 
 -0.019873 
 -0.016447 
 
 
 densenet201-bert-lstm 
 0.002938 
 -0.020161 
 -0.021564 
 
 
 densenet201-bert-pos-lstm 
 0.003525 
 -0.021820 
 -0.021775 
 
 
 densenet201-bert-ner-lstm 
 -0.002350 
 -0.031391 
 -0.026532 
 
 
 densenet201-bert-pos-ner-lstm 
 -0.008226 
 -0.037070 
 -0.027623

# Drafts

In [30]:
# import shutil
# def remove_folder(path):
#     # check if folder exists
#     if os.path.exists(path):
#          # remove if exists
#          shutil.rmtree(path)
#     else:
#          # throw your exception to handle this special scenario
#          raise XXError("your exception") 
# remove_folder("./model_checkpoint")
# remove_folder("./model_history")

In [31]:
# ## Choose best
# # Load text feature
# mvsa_single_bert, mvsa_multiple_bert = load_mvsa_feature('bert-base')
# mvsa_single_pos_bow, mvsa_multiple_pos_bow = load_mvsa_feature('pos-bow')
# mvsa_single_pos_tfidf, mvsa_multiple_pos_tfidf = load_mvsa_feature('pos-tfidf')
# mvsa_single_ner_bow, mvsa_multiple_ner_bow = load_mvsa_feature('ner-bow')
# mvsa_single_ner_tfidf, mvsa_multiple_ner_tfidf = load_mvsa_feature('ner-tfidf')

# ## Load image feature
# mvsa_single_vgg16, mvsa_multiple_vgg16 = load_mvsa_feature('vgg16')
# mvsa_single_vgg19, mvsa_multiple_vgg19 = load_mvsa_feature('vgg19')
# mvsa_single_resnet50, mvsa_multiple_resnet50 = load_mvsa_feature('resnet50')
# mvsa_single_resnet101, mvsa_multiple_resnet101 = load_mvsa_feature('resnet101')
# mvsa_single_resnet152, mvsa_multiple_resnet152 = load_mvsa_feature('resnet152')
# mvsa_single_densenet121, mvsa_multiple_densenet121 = load_mvsa_feature('densenet121')
# mvsa_single_densenet169, mvsa_multiple_densenet169 = load_mvsa_feature('densenet169')
# mvsa_single_densenet201, mvsa_multiple_densenet201 = load_mvsa_feature('densenet201')

# mvsa_single_bert_pos = np.concatenate((mvsa_single_bert, mvsa_single_pos_tfidf), axis=1)
# mvsa_single_bert_ner = np.concatenate((mvsa_single_bert, mvsa_single_ner_tfidf), axis=1)
# mvsa_single_bert_pos_ner = np.concatenate((mvsa_single_bert, mvsa_single_pos_tfidf, mvsa_single_ner_tfidf), axis=1)

# mvsa_multiple_bert_pos = np.concatenate((mvsa_multiple_bert, mvsa_multiple_pos_tfidf), axis=1)
# mvsa_multiple_bert_ner = np.concatenate((mvsa_multiple_bert, mvsa_multiple_ner_tfidf), axis=1)
# mvsa_multiple_bert_pos_ner = np.concatenate((mvsa_multiple_bert, mvsa_multiple_pos_tfidf, mvsa_multiple_ner_tfidf), axis=1)

In [32]:
# # save shuffle indices for other experiments consistency (temporary fix)
# random_idx_single = np.random.permutation(len(mvsa_single_multimodal_labels))
# random_idx_multiple = np.random.permutation(len(mvsa_multiple_multimodal_labels))
# np.save('mvsa-single-shuffle-indices.npy', random_idx_single)
# np.save('mvsa-multiple-shuffle-indices.npy', random_idx_multiple)

In [33]:
# scores_text = []
# for i in range(len(feature_names)):
#     print('MVSA-Single:', feature_names[i])
#     if 'lstm' in feature_names[i]:
#         score_text = run_and_evaluate_LF_new('single-LF-' + feature_names[i],
#                                                                 mvsa_single_features[i][0], 
#                                                                 mvsa_single_features[i][1],
#                                                                 mvsa_single_multimodal_labels, 
#                                                                 verbose=0)
#     else:
#         score_text = run_and_evaluate_LF_new('single-LF-' + feature_names[i],
#                                                                 mvsa_single_features[i][0], 
#                                                                 mvsa_single_features[i][1],
#                                                                 mvsa_single_multimodal_labels, 
#                                                                 verbose=0, lstm=False)    
#     scores_text.append(score_text)
# #     scores_image.append(score_image)
# #     scores_LF.append(score_LF)
#     print()

In [34]:
# df1_single_scores_text = pd.DataFrame(scores_text, columns=['Accuracy', 'F1-macro', 'F1-weighted'], index=textual_feature_names)
# style_dataframe(df1_single_scores_text)

In [35]:
# scores_text0 = []
# for i in range(len(feature_names)):
#     print('MVSA-Single:', feature_names[i])
#     if 'lstm' in feature_names[i]:
#         score_text = run_and_evaluate('single-LF-' + feature_names[i],
#                                                                 mvsa_single_features[i][0], 
#                                                                 mvsa_single_features[i][1],
#                                                                 mvsa_single_multimodal_labels, 
#                                                                 verbose=0)[1]
#     else:
#         score_text = run_and_evaluate('single-LF-' + feature_names[i],
#                                                                 mvsa_single_features[i][0], 
#                                                                 mvsa_single_features[i][1],
#                                                                 mvsa_single_multimodal_labels,
#                                                                 verbose=0, lstm=False)[1]  
#     scores_text0.append(score_text)
# #     scores_image.append(score_image)
# #     scores_LF.append(score_LF)
#     print()

In [36]:
# # sing
# df0_single_scores_text = pd.DataFrame(scores_text0, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=textual_feature_names)
# style_dataframe(df0_single_scores_text)

In [37]:
# scores_text_0m = []
# for i in range(len(feature_names)):
#     print('MVSA-Multiple:', feature_names[i])
#     if 'lstm' in feature_names[i]:
#         score_text = run_and_evaluate('multiple-LF-' + feature_names[i],
#                                                                 mvsa_multiple_features[i][0], 
#                                                                 mvsa_multiple_features[i][1],
#                                                                 mvsa_multiple_multimodal_labels, 
#                                                                 verbose=0)[1]
#     else:
#         score_text = run_and_evaluate('multiple-LF-' + feature_names[i],
#                                                                 mvsa_multiple_features[i][0], 
#                                                                 mvsa_multiple_features[i][1],
#                                                                 mvsa_multiple_multimodal_labels,
#                                                                 verbose=0, lstm=False)[1]  
#     scores_text_0m.append(score_text)
# #     scores_image.append(score_image)
# #     scores_LF.append(score_LF)
#     print()

In [38]:
# # mul
# df0_multiple_scores_text = pd.DataFrame(scores_text_0m, columns=['Loss', 'Accuracy', 'F1-macro', 'F1-weighted'], index=textual_feature_names)
# style_dataframe(df0_multiple_scores_text)

In [39]:
# def run_and_evaluate(name, X, X2, y, verbose=0, lstm=True):
#     data = preprocess_inputs(X, X2, y)
#     X_train, X_val, X_test = data['texts']
#     X2_train, X2_val, X2_test = data['images']
#     y_train, y_val, y_test = data['labels']
    
#     model = create_model_text(X_train.shape[1:], lstm=lstm)
#     early_stopping = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=EARLY_STOPPING)
#     checkpoint = ModelCheckpoint('./model_checkpoint/test-{}.h5'.format(name), save_best_only=True, verbose=verbose)
    
#     reset_seeds()
#     history = model.fit(X_train, y_train, validation_data=(X_val, y_val), 
#                         epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=verbose,
#                         callbacks=[checkpoint, early_stopping])
#     if verbose == 1:
#         best_epoch = np.argmin(history.history['val_loss'])
#         print('Checkpoint loaded at epoch:', best_epoch)
    
#     return history, evaluate_model(model, X_test, y_test, checkpoint=name, verbose=verbose)

# def evaluate_model(model, X_test, y_test, checkpoint=None, verbose=1):
#     if checkpoint is not None:
#         model = load_model('./model_checkpoint/test-{}.h5'.format(checkpoint))
#     loss, acc, f1_macro, f1_weighted = model.evaluate(X_test, y_test, verbose=verbose)
    
#     if verbose == 1:
#         print('Loss:', loss)
#         print('Accuracy:', acc)
#         print('Macro F1-score:', f1_macro)
#         print('Weighted F1-score:', f1_weighted)

#     return loss, acc, f1_macro, f1_weighted

In [40]:
# def run_and_evaluate_LF_new(name, X1, X2, y, verbose=0, lstm=True):

#     data = preprocess_inputs(X1, X2, y)
#     X1_train, X1_val, X1_test = data['texts']
#     X2_train, X2_val, X2_test = data['images']
#     y_train, y_val, y_test = data['labels']

#     early_stopping_text = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=EARLY_STOPPING)
#     early_stopping_image = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=EARLY_STOPPING)

#     if 'multiple' in name:
#         batch_size = 256
#     else:
#         batch_size = BATCH_SIZE # 128

#     checkpoint_text_path = './model_checkpoint/{}.h5'.format(name.split('-')[0] + '-' + '-'.join(name.split('-')[3:]))
#     checkpoint_image_path = './model_checkpoint/{}.h5'.format(name.split('-')[0] + '-' + name.split('-')[2])

#     history_text_path = './model_history/{}.npy'.format(name.split('-')[0] + '-' + '-'.join(name.split('-')[3:]))
#     history_image_path = './model_history/{}.npy'.format(name.split('-')[0] + '-' + name.split('-')[2])

#     if lstm == True:
#         checkpoint_image_path = './model_checkpoint/{}.h5'.format(name.split('-')[0] + '-' + name.split('-')[2] + '-lstm')
#         history_image_path = './model_history/{}.npy'.format(name.split('-')[0] + '-' + name.split('-')[2] + '-lstm')
    
#     if not (os.path.exists(checkpoint_text_path) and os.path.exists(history_text_path)):
#         print('Create new text model:', os.path.split(checkpoint_text_path)[1])

#         model_text = create_model_text(X1_train.shape[1:], lstm=lstm)
#         checkpoint_text = ModelCheckpoint(checkpoint_text_path, save_best_only=True, verbose=verbose)
#         history_text = model_text.fit(X1_train, y_train, validation_data=(X1_val, y_val), 
#                                   epochs=EPOCHS, batch_size=batch_size, verbose=verbose,
#                                   callbacks=[checkpoint_text, early_stopping_text])
#         if not os.path.exists(os.path.split(history_text_path)[0]):
#             os.makedirs(os.path.split(history_text_path)[0])
#         pickle.dump(history_text.history, open(history_text_path, 'wb'))
    
# #     if not(os.path.exists(checkpoint_image_path) and os.path.exists(history_image_path)):
# #         print('Create new image model:', os.path.split(checkpoint_image_path)[1])

# #         model_image = create_model_image(X2_train.shape[1:], lstm=lstm)
# #         checkpoint_image = ModelCheckpoint(checkpoint_image_path, 
# #                                        save_best_only=True, verbose=verbose)
# #         history_image = model_image.fit(X2_train, y_train, validation_data=(X2_val, y_val), 
# #                                 epochs=EPOCHS, batch_size=batch_size, verbose=verbose,
# #                                 callbacks=[checkpoint_image, early_stopping_image])
# #         pickle.dump(history_image.history, open(history_image_path, 'wb'))
    
#     model_text = load_model(checkpoint_text_path)
# #     model_image = load_model(checkpoint_image_path)

#     history_text = pickle.load(open(history_text_path, 'rb'))    
# #     history_image = pickle.load(open(history_image_path, 'rb'))

#     y_pred_text = model_text.predict(X1_test)
# #     y_pred_image = model_image.predict(X2_test)

#     best_epoch_text = np.argmin(history_text['val_loss'])
# #     best_epoch_image = np.argmin(history_image['val_loss'])
#     print('Model text checkpoint loaded at epoch:', best_epoch_text)
# #     print('Model image checkpoint loaded at epoch:', best_epoch_image)
    
# #     val_acc_text = history_text['val_accuracy'][best_epoch_text]
# #     val_acc_image = history_image['val_accuracy'][best_epoch_image]
# #     weights = get_average_weights(val_acc_text, val_acc_image)
# #     print('Model weights (text, image):', weights)

# #     y_pred = weighted_average(weights, np.asarray([y_pred_text, y_pred_image], dtype='float32'))
    
#     eval_text = evaluate_model_uni(model_text, X1_test, y_test, verbose=verbose)
# #     eval_image = evaluate_model_uni(model_image, X2_test, y_test, verbose=verbose)
# #     eval_LF = evaluate_model_LF(y_test, y_pred, verbose=verbose)
#     return eval_text#, eval_image, eval_LF